In [1]:
from solcore import si, material
from solcore.structure import Layer, Structure, Junction
from solcore.solar_cell import SolarCell
import solcore.quantum_mechanics as QM
import solcore.poisson_drift_diffusion as PDD
import numpy as np
from solcore.light_source import LightSource
from solcore.solar_cell_solver import solar_cell_solver
import matplotlib.pyplot as plt
import os
import shutil

In [2]:
wl = np.linspace(350, 1200, 401) * 1e-9
light_source = LightSource(
    source_type="standard",
    version="AM1.5g",
    x=wl,
    output_units="photon_flux_per_m",
    concentration=1,
)
bulk = material("GaAs")(T=293, strained=False)
barrier = material("GaAsP")(T=293, P=0.1, strained=True)

# As well as some of the layers
top_layer = Layer(width=si("30nm"), material=barrier)
inter = Layer(width=si("3nm"), material=bulk)
barrier_layer = Layer(width=si("5nm"), material=barrier)
bottom_layer = top_layer
wl = np.linspace(350, 1050, 301) * 1e-9

InSb = material('InSb')(T=293, strained=False)
GaSb = material('GaSb')(T=293, strained=False)
buffer = material('AlGaAs')(T=293, Al=0.3)
capping = material('GaAs')(T=293)
n_GaAs = material('GaAs')(T=293, Nd=1e15)
p_GaAs = material('GaAs')(T=293, Na=1e15)
# We create the QW material at the given composition
QW = material("InGaAs")(T=293, In=0.15, strained=True)

n_layer = Layer(width=250e-9, material=n_GaAs)
QW1 = Layer(width=5e-9, material=InSb)
Capping_layer = Layer(width=80e-9, material=capping)
QW2 = Layer(width=5e-9, material=GaSb)
p_layer = Layer(width=400e-9, material=p_GaAs)
test_ver1 = Structure([n_layer, QW1, Capping_layer, QW2, p_layer], substrate=bulk)
test_ver2 = Structure([n_layer, QW1, Capping_layer, p_layer], substrate=bulk)

In [41]:
def showeiei(barrier, interlayer, dot):
    T = 293
    n_GaAs = material("GaAs")(T=T, Nd=1e24)
    p_GaAs = material("GaAs")(T=T, Na=8e22)
    QWmat = material("InGaAs")(T=T, In=0.2, strained=True)
    Bmat = material("GaAsP")(T=T, P=0.1, strained=True)
    i_GaAs = material("GaAs")(T=T)
    i_GaAs_dope = material("GaAs")(T=T, Na= 1e14)

    struc = Structure([Layer(width=50e-9, material=n_GaAs, role="Emitter")]+
                      [Layer(width=barrier, material=Bmat, role="barrier")] +
                      + 10 * [Layer(width=barrier, material=Bmat, role="barrier"),
                              Layer(width=interlayer, material=i_GaAs_dope, role="interlayer"),
                              Layer(width=dot, material=QWmat, role="well"),
                              Layer(width=interlayer, material=i_GaAs_dope, role="interlayer"),
                              Layer(width=barrier, material=Bmat, role="barrier")] +
                      # +10 * [Layer(width=10e-9, material=Bmat, role="barrier"),
                             # Layer(width=2e-9, material=i_GaAs, role="well"),
                             # Layer(width=7e-9, material=QWmat, role="well"),
                             # Layer(width=2e-9, material=i_GaAs, role="well"),
                             # Layer(width=10e-9, material=Bmat, role="barrier"),] +
                      [Layer(width=barrier, material=Bmat, role="barrier")] +
                      [Layer(width=50e-9, material=p_GaAs, role="Base")],
                      substrate=i_GaAs)
    # print(struc)
    # a = QM.structure_utilities.text_render(struc, resolution=100)
    # print(a)
    output_2 = QM.schrodinger(struc, quasiconfined=0.05,kpoints=40,
                               num_eigenvalues=200,plot_bands=True,symmetric=True
                               )
    # print(output_2)
    for i in output_2:
        print(i)
        print('')
    return output_2
   

In [42]:
output_2 = showeiei(5e-9,5e-9,5e-9)

{'x': array([0.00000000e+00, 2.50173732e-10, 5.00347464e-10, ...,
       3.59499653e-07, 3.59749826e-07, 3.60000000e-07]), 'potentials': {'x': array([0.00000000e+00, 2.50173732e-10, 5.00347464e-10, ...,
       3.59499653e-07, 3.59749826e-07, 3.60000000e-07]), 'Ve': array([1.00238096e-19, 1.00238096e-19, 1.00238096e-19, ...,
       1.00238096e-19, 1.00238096e-19, 1.00238096e-19]), 'Vhh': array([-1.28174117e-19, -1.28174117e-19, -1.28174117e-19, ...,
       -1.28174117e-19, -1.28174117e-19, -1.28174117e-19]), 'Vlh': array([-1.28174117e-19, -1.28174117e-19, -1.28174117e-19, ...,
       -1.28174117e-19, -1.28174117e-19, -1.28174117e-19]), 'Vso': array([-1.82808334e-19, -1.82808334e-19, -1.82808334e-19, ...,
       -1.82808334e-19, -1.82808334e-19, -1.82808334e-19])}, 'effective_masses': {'x': array([0.00000000e+00, 2.50173732e-10, 5.00347464e-10, ...,
       3.59499653e-07, 3.59749826e-07, 3.60000000e-07]), 'me': array([6.85021545e-32, 6.85021545e-32, 6.85021545e-32, ...,
       6.85021545

In [ ]:
QM.graphics.schrodinger_graph(output_2[2])
QM.graphics.potentials_graph(output_2[2], Ve=None, Vhh=None, Vlh=None, color='grey',)
QM.graphics.split_schrodinger_graph_LDOS(output_2[2])

QM.graphics.split_schrodinger_graph_potentials(output_2[2], trim_levels_beyond=0.01, linewidth=1,scale=0.3, suppress_invert=False, probability_density=False,wfalpha=0.8, potentialalpha=0.8)
    

In [ ]:
def solar_cells(barrier, interlayer, dot,round):

    T = 298
    wl = np.linspace(350, 1050, 301) * 1e-9

    # First, we create the materials of the QW
    # สร้างวัสดุทำรับQW ในsolar cell
    QWmat = material("InGaAs")(T=T, In=0.2, strained=True)
    Bmat = material("GaAsP")(T=T, P=0.1, strained=True)
    i_GaAs = material("GaAs")(T=T)
    i_GaAs_dope = material("GaAs")(T=T, Na=1e14)
    QW = PDD.QWunit([
        Layer(width=barrier, material=Bmat, role="barrier"),
        Layer(width=interlayer, material=i_GaAs_dope, role="well"),
        Layer(width=dot, material=QWmat, role="well"),
        Layer(width=interlayer, material=i_GaAs_dope, role="well"),
        Layer(width=barrier, material=Bmat, role="barrier")
    ],T=T,repeat=30,substrate=i_GaAs,)
    QW_list = QW.GetEffectiveQW(wavelengths=wl)
    n_GaAs = material("GaAs")(T=T, Nd=1e17)
    p_GaAs = material("GaAs")(T=T, Na=8e17)
    GaAs_junction = Junction(
        [Layer(width=150e-9, material=n_GaAs, role="Emitter"),
         Layer(width=barrier, material=Bmat, role="barrier")]
        + QW_list
        + [Layer(width=barrier, material=Bmat, role="barrier"),
           Layer(width=2000e-9, material=p_GaAs, role="Base"),
        ],
        sn=1e6,
        sp=1e6,T=T,kind="PDD",)
    MgF2 = material("MgF2")()
    ZnS = material("ZnScub")()
    my_solar_cell = SolarCell([Layer(width=110e-9, material=MgF2, role="ARC1"),Layer(width=60e-9, material=ZnS, role="ARC2"),GaAs_junction,],T=T,substrate=n_GaAs,)
    solar_cell_solver(my_solar_cell, "qe",user_options={"light_source": light_source,"wavelength": wl, "optics_method": "TMM",}, )
    num_con = 5  # จำนวนในการสร้างแสง
    con = np.logspace(0, 3, num_con);vint = np.linspace(-3.5, 4, 600)
    V = np.linspace(-3.5, 0, 300)
    allI = []; isc = []; voc = []; FF = []; pmpp = []
    fig3, axIV = plt.subplots(1, 1, figsize=(6, 4))
    for i,c in enumerate(con):  # ทำการยิงแสงทั้งหมด20ครั้งตามcon
        light_source.concentration = c
        solar_cell_solver(my_solar_cell,"iv",user_options={"light_source": light_source,"wavelength": wl,"optics_method": None,"light_iv": True,  "mpp": True, "voltages": V,"internal_voltages": vint,  },)
        isc.append(my_solar_cell.iv["Isc"])
        voc.append(my_solar_cell.iv["Voc"])
        FF.append(my_solar_cell.iv["FF"])
        pmpp.append(my_solar_cell.iv["Pmpp"])
        allI.append(my_solar_cell.iv["IV"][1])
        axIV.plot(-V, my_solar_cell.iv["IV"][1] / isc[-1], label=int(c))  # เป็นการวัดลักษณะIVเมิอแสงเปลี่ยนไป
        print('======================================================================')
        print(f'{2*i+round*10}%',f'{round}/10',i)
        print('======================================================================')

    axIV.legend(loc="lower left", frameon=False)
    axIV.set_ylim(0, 1.1)
    axIV.set_xlim(0, 3.5)
    axIV.set_xlabel("Voltage (V)")
    axIV.set_ylabel("Normalized current (-)")
    plt.tight_layout()

    fig2, axes = plt.subplots(2, 2, figsize=(11.25, 8))

    axes[0, 0].semilogx(con, np.array(pmpp) / con / 10,
                        "r-o")  # ทำไมต้องหาร10ด้วยไม่เข้าใจเลย(ไม่ใช้หารด้วย 0.97หรอเพราะว่าAM1.5g = 970w/m^2)แล้วทำไมไม่x100
    axes[0, 0].set_xlabel("Concentration (suns)")
    axes[0, 0].set_ylabel("Efficiency (%)")

    axes[0, 1].loglog(con, abs(np.array(isc)), "b-o")
    axes[0, 1].set_xlabel("Concentration (suns)")
    axes[0, 1].set_ylabel("I$_{SC}$ (Am$^{-2}$)")

    axes[1, 0].semilogx(con, abs(np.array(voc)), "g-o")
    axes[1, 0].set_xlabel("Concentration (suns)")
    axes[1, 0].set_ylabel("V$_{OC}$ (V)")

    axes[1, 1].semilogx(con, abs(np.array(FF)) * 100, "k-o")
    axes[1, 1].set_xlabel("Concentration (suns)")
    axes[1, 1].set_ylabel("Fill Factor (%)")

    plt.tight_layout()

    # We can plot the electron and hole densities in equilibrium and at short circuit
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(11.25, 4))
    for j in my_solar_cell.junction_indices:  # junctionคือหยั่ง
        zz = (
                my_solar_cell[j].short_circuit_data.Bandstructure["x"] + my_solar_cell[j].offset)
        n = my_solar_cell[j].short_circuit_data.Bandstructure["n"]
        p = my_solar_cell[j].short_circuit_data.Bandstructure["p"]
        ax1.semilogy(zz * 1e9, n, "b")  # อันนี้น่าจะเป็นการระบุcarrier densityของตอนฉายแสงของe และhole
        ax1.semilogy(zz * 1e9, p, "r")

        zz = my_solar_cell[j].equilibrium_data.Bandstructure["x"] + my_solar_cell[j].offset
        n = my_solar_cell[j].equilibrium_data.Bandstructure["n"]
        p = my_solar_cell[j].equilibrium_data.Bandstructure["p"]
        ax1.semilogy(zz * 1e9, n, "b--")  # อันนี้น่าจะเป็นการระบุcarrier densityของตอนปิดแสงของe และhole
        ax1.semilogy(zz * 1e9, p, "r--")

    ax1.set_xlabel("Position (nm)")
    ax1.set_ylabel("Carrier density (m$^{-3}$)")
    plt.tight_layout()

    # And we plot the QE
    labels = ["EQE GaInP", "EQE GaAs"]
    colours = ["b", "r"]
    for i, j in enumerate(my_solar_cell.junction_indices):  # เป็นการพล็อตของeqeของแต่ละอันของGaAs และGaInP
        ax2.plot(wl * 1e9, my_solar_cell[j].eqe(wl), colours[i], label=labels[i])

    ax2.plot(wl * 1e9, my_solar_cell.absorbed, "k", label="Total Absorbed")

    ax2.legend(loc="upper right", frameon=False)
    ax2.set_xlabel("Wavelength (nm)")
    ax2.set_ylabel("EQE")
    ax2.set_ylim(0, 1.1)
    ax2.set_xlim(350, 1150)

    plt.tight_layout()
    fig3.savefig(f'IV_curve_QW_{dot:.2e}_width.svg', dpi=300)
    fig2.savefig(f'performance_QW_{dot:.2e}_width.svg', dpi=300)
    fig.savefig(f'EQE_QW_{dot:.2e}_width.svg', dpi=300)
    text = ''
    data = save_tuple2text(text,'profile'+str(dot)+'width', ('Profile', f'''[Layer(width=150e-9, material=n_GaAs, role="Emitter")\n 
    Layer(width={barrier:.2e}, material=Bmat, role="barrier")]\n
    Layer(width={barrier:.2e}, material=Bmat, role="barrier"),\n
    Layer(width={interlayer:.2e}, material=i_GaAs_dope, role="well"),\n
    Layer(width={dot:.2e}, material=QWmat, role="well"),\n
    Layer(width={interlayer:.2e}, material=i_GaAs_dope, role="well"),\n
    Layer(width={barrier:.2e}, material=Bmat, role="barrier")\n
    Layer(width={barrier:.2e}, material=Bmat, role="barrier"),\n
    Layer(width=2000e-9, material=p_GaAs, role="Base"),\n'''))
    save_file_direction(f'data_of_dot_width{dot:.2e}', data, f'QWcapping_width{dot:.2e}')
    current_path = os.getcwd()
    save_path = os.path.join(current_path, f'data_of_dot_width{dot:.2e}')
    fig1_loc = os.path.join(current_path, f'EQE_QW_{dot:.2e}_width.svg')
    fig1_loc_new = os.path.join(save_path, f'EQE_QW_{dot:.2e}_width.svg')
    shutil.move(fig1_loc, fig1_loc_new)
    fig2_loc = os.path.join(current_path, f'performance_QW_{dot:.2e}_width.svg')
    fig2_loc_new = os.path.join(save_path, f'performance_QW_{dot:.2e}_width.svg')
    shutil.move(fig2_loc, fig2_loc_new)
    fig3_loc = os.path.join(current_path, f'IV_curve_QW_{dot:.2e}_width.svg')
    fig3_loc_new = os.path.join(save_path, f'IV_curve_QW_{dot:.2e}_width.svg')
    shutil.move(fig3_loc, fig3_loc_new)
    print('======================================================================')
    print(f'{2 * i + round * 10}%', f'{round}/10', i, 'save susses')
    print('======================================================================')
